# 贴现记录表分数

In [2]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def suanqingdan():
    global zongfen
    df=pd.read_excel('计算分数/2018-2019年6月业务.xls',encoding='GB18030')
    df=df[['票面金额(元)','交易机构','当前状态','交易对手','年份']]
    df['票面金额(元)']=df['票面金额(元)'].astype('float')
    

    
#1.贴现量分数    
    liang=df['票面金额(元)'].groupby([df['年份'],df['交易机构']]).sum().unstack('年份')
    liang['2018贴现量上半年排名']=liang['2018上半年'].rank(ascending=False)
    liang['2018贴现量上半年排名']=liang['2018贴现量上半年排名'].astype('int')
    liang['2018贴现量下半年排名']=liang['2018年下半年'].rank(ascending=False)
    liang['2018贴现量下半年排名']=liang['2018贴现量下半年排名'].astype('int')
    liang['2019贴现量上半年排名']=liang['2019年上半年'].rank(ascending=False)
    liang['2019贴现量上半年排名']=liang['2019贴现量上半年排名'].astype('int')    
    
    
    liang['2018贴现量上半年排名得分']=liang['2018贴现量上半年排名'].replace(9,999).replace(1,15).replace(2,13).replace(3,11).replace(4,9).replace(5,7).replace(6,7).replace(8,7).replace(999,5).replace(10,5)
    liang['2018贴现量下半年排名得分']=liang['2018贴现量下半年排名'].replace(9,999).replace(1,15).replace(2,13).replace(3,11).replace(4,9).replace(5,7).replace(6,7).replace(8,7).replace(999,5).replace(10,5)
    liang['2019贴现量上半年排名得分']=liang['2019贴现量上半年排名'].replace(9,999).replace(1,15).replace(2,13).replace(3,11).replace(4,9).replace(5,7).replace(6,7).replace(8,7).replace(999,5).replace(10,5)

    
    
    liang['2019年贴现量上半年环比增长率']=(liang['2019年上半年']-liang['2018年下半年'])/liang['2018年下半年']
    liang['2018年贴现量下半年环比增长率']=(liang['2018年下半年']-liang['2018上半年'])/liang['2018上半年']


    liang= liang.reset_index(drop=False)    #重新定义索引

    for i in range(0,len(liang)):
        if liang.loc[i,'2019年贴现量上半年环比增长率']>0.03:
            liang.loc[i,'2019年贴现量上半年环比增长率得分']=3
        elif liang.loc[i,'2019年贴现量上半年环比增长率']<0:
            liang.loc[i,'2019年贴现量上半年环比增长率得分']=0
        else:
            liang.loc[i,'2019年贴现量上半年环比增长率得分']=0
    for i in range(0,len(liang)):
        if liang.loc[i,'2018年贴现量下半年环比增长率']>0.03:
            liang.loc[i,'2018年贴现量下半年环比增长率得分']=3
        elif liang.loc[i,'2018年贴现量下半年环比增长率']< 0:
            liang.loc[i,'2018年贴现量下半年环比增长率得分']=0
        else:
            liang.loc[i,'2018年贴现量下半年环比增长率得分']=0
        
    liang.to_excel('贴现量分数.xls')
    liang=liang.set_index(['交易机构'])

    
 #2.客户数量排名







    kehuzhong1=df[df['年份']=='2018上半年']
    kehuzhong1=kehuzhong1['票面金额(元)'].groupby([kehuzhong1['交易机构'],kehuzhong1['交易对手']]).sum()
    kehuzhong1= kehuzhong1.reset_index(drop=False)    #重新定义索引
    kehuzhong1['年份']='2018上半年'
    
    kehuzhong2=df[df['年份']=='2018年下半年']
    kehuzhong2=kehuzhong2['票面金额(元)'].groupby([kehuzhong2['交易机构'],kehuzhong2['交易对手']]).sum()
    kehuzhong2= kehuzhong2.reset_index(drop=False)    #重新定义索引
    kehuzhong2['年份']='2018年下半年'

    
    kehuzhong3=df[df['年份']=='2019年上半年']
    kehuzhong3=kehuzhong3['票面金额(元)'].groupby([kehuzhong3['交易机构'],kehuzhong3['交易对手']]).sum()
    kehuzhong3= kehuzhong3.reset_index(drop=False)  
    kehuzhong3['年份']='2019年上半年'

  #  kehuzhong1=kehuzhong1.drop_duplicates(['交易对手'],keep='first')   #删除连接中的重复行
  #  print(len(kehuzhong1))
  #  kehuzhong2=df[df['年份']=='2018年下半年']
  #  kehuzhong2=kehuzhong2.drop_duplicates(['交易对手'],keep='first')   #删除连接中的重复行
  #  print(len(kehuzhong2))

  #  kehuzhong3=df[df['年份']=='2019年上半年']
  #  kehuzhong3=kehuzhong3.drop_duplicates(['交易对手'],keep='first')   #删除连接中的重复行
  #  print(len(kehuzhong3))

    kehu=pd.concat([kehuzhong1,kehuzhong2,kehuzhong3])
   # kehu=kehu[kehu['票面金额(元)']>=1000000 ]                 #半年贴现金额大于1000000
  #  print(kehu)

    kehu=kehu['交易对手'].groupby([kehu['年份'],kehu['交易机构']]).size().unstack('年份')
    print(kehu)
    
    kehu['2018客户数量上半年排名']=kehu['2018上半年'].rank(ascending=False).astype('int')
    kehu['2018客户数量下半年排名']=kehu['2018年下半年'].rank(ascending=False).astype('int')
    kehu['2019客户数量上半年排名']=kehu['2019年上半年'].rank(ascending=False).astype('int')
    kehu['2018客户数量上半年排名得分']=kehu['2018客户数量上半年排名'].replace(9,999).replace(1,15).replace(2,15).replace(3,12).replace(4,12).replace(5,9).replace(6,9).replace(7,9).replace(8,6).replace(999,6).replace(10,3)
    kehu['2018客户数量下半年排名得分']=kehu['2018客户数量下半年排名'].replace(9,999).replace(1,15).replace(2,15).replace(3,12).replace(4,12).replace(5,9).replace(6,9).replace(7,9).replace(8,6).replace(999,6).replace(10,3)
    kehu['2019客户数量上半年排名得分']=kehu['2019客户数量上半年排名'].replace(9,999).replace(1,15).replace(2,15).replace(3,12).replace(4,12).replace(5,9).replace(6,9).replace(7,9).replace(8,6).replace(999,6).replace(10,3)

    
    kehu['2019年客户数量上半年环比增长率']=(kehu['2019年上半年']-kehu['2018年下半年'])/kehu['2018年下半年']
    kehu['2018年客户数量下半年环比增长率']=(kehu['2018年下半年']-kehu['2018上半年'])/kehu['2018上半年']
    
    
    kehu= kehu.reset_index(drop=False)    #重新定义索引

    
    for i in range(0,len(kehu)):
        if kehu.loc[i,'2019年客户数量上半年环比增长率']>0.03:
            kehu.loc[i,'2019年客户数量上半年环比增长率得分']=3
        elif kehu.loc[i,'2019年客户数量上半年环比增长率']<0:
            kehu.loc[i,'2019年客户数量上半年环比增长率得分']=0
        else:
            kehu.loc[i,'2019年客户数量上半年环比增长率得分']=0
            
    for i in range(0,len(kehu)):
        if kehu.loc[i,'2018年客户数量下半年环比增长率']>0.03:
            kehu.loc[i,'2018年客户数量下半年环比增长率得分']=3
        elif kehu.loc[i,'2018年客户数量下半年环比增长率']< 0:
            kehu.loc[i,'2018年客户数量下半年环比增长率得分']=0
        else:
            kehu.loc[i,'2018年客户数量下半年环比增长率得分']=0
    
    
    
    kehu.to_excel('客户数量分数.xls')
    kehu=kehu.set_index(['交易机构'])

#2.新客户数量排名
#2.1  2019年上半年新客户
    xinkehu2019=df[(df['年份']=='2018年下半年')|(df['年份']=='2019年上半年')]
    xinkehu2019=xinkehu2019.sort_values(by='年份',ascending=True)
    xinkehu2019= xinkehu2019.reset_index(drop=True)    #重新定义索引
    xinkehu2019=xinkehu2019.drop_duplicates(['交易对手'],keep='first')   #删除连接中的重复行
              
    xinkehu2019=xinkehu2019[xinkehu2019['年份']=='2019年上半年']
    xinkehu2019=xinkehu2019['交易对手'].groupby([xinkehu2019['年份'],xinkehu2019['交易机构']]).size().unstack('年份')
                        
    xinkehu2018xia=df[(df['年份']=='2018年下半年')|(df['年份']=='2018上半年')]
    xinkehu2018xia=xinkehu2018xia.sort_values(by='年份',ascending=True)
    xinkehu2018xia= xinkehu2018xia.reset_index(drop=True)    #重新定义索引
    xinkehu2018xia=xinkehu2018xia.drop_duplicates(['交易对手'],keep='first')   #删除连接中的重复行
    xinkehu2018xia=xinkehu2018xia[xinkehu2018xia['年份']=='2018年下半年']
    xinkehu2018xia=xinkehu2018xia['交易对手'].groupby([xinkehu2018xia['年份'],xinkehu2018xia['交易机构']]).size().unstack('年份')
    xinkehu=pd.concat([xinkehu2019,xinkehu2018xia], axis=1)  
    
    xinkehu['2019年上半年新增客户排名']=xinkehu['2019年上半年'].rank(ascending=False).astype('int')
    xinkehu['2018年下半年新增客户排名']=xinkehu['2018年下半年'].rank(ascending=False).astype('int')
    xinkehu['2019年上半年新增客户排名得分']=xinkehu['2019年上半年新增客户排名'].replace(10,1000).replace(8,800).replace(1,10).replace(2,10).replace(3,8).replace(4,8).replace(5,6).replace(7,6).replace(800,4).replace(9,4).replace(1000,2)
    xinkehu['2018年下半年新增客户排名得分']=xinkehu['2018年下半年新增客户排名'].replace(10,1000).replace(8,800).replace(1,10).replace(2,10).replace(3,8).replace(4,8).replace(5,6).replace(7,6).replace(800,4).replace(9,4).replace(1000,2)

    
    
    xinkehu['2019年新客户数量上半年环比增长率']=(xinkehu['2019年上半年']-xinkehu['2018年下半年'])/xinkehu['2018年下半年']
   # xinkehu['2018年新客户数量下半年环比增长率']=(xinkehu['2018年下半年']-xinkehu['2018上半年'])/xinkehu['2018上半年']

    xinkehu= xinkehu.reset_index(drop=False)    #重新定义索引

    
    for i in range(0,len(xinkehu)):
        if xinkehu.loc[i,'2019年新客户数量上半年环比增长率']>0.03:
            xinkehu.loc[i,'2019年新客户数量上半年环比增长率得分']=2
        elif xinkehu.loc[i,'2019年新客户数量上半年环比增长率']< 0:
            xinkehu.loc[i,'2019年新客户数量上半年环比增长率得分']=0
        else:
            xinkehu.loc[i,'2019年新客户数量上半年环比增长率得分']=0
            
  
    
                  
                            
    xinkehu.to_excel('新客户数量分数.xls')
    xinkehu=xinkehu.set_index(['交易机构'])
             
   # biao=biao.set_index(['发行日', '期限','银行分类'])
        
    
    zongfen=pd.concat([liang,kehu,xinkehu], axis=1)  
    zongfen['来源']='余额明细'
    zongfen.to_excel('总分.xls')
    
    
suanqingdan()



年份     2018上半年  2018年下半年  2019年上半年
交易机构                              
丽水市分行       32        31        27
台州市分行       47        54        57
嘉兴市分行       25        27        25
杭州市分行       33        63        61
温州市分行       40        55        47
湖州市分行       21        17        18
绍兴市分行       57        84        90
舟山市分行        8         8        21
衢州市分行       12        13        12
金华市分行       19        47        30


# 再贴现分数 

In [3]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def zaitiexian():
    global df
    df=pd.read_excel('计算分数\再贴现分数统计.xlsx',encoding='GB18030')
    
    for i in range(0,len(df)):
        if df.loc[i,'2018年上半年']==0:
            df.loc[i,'2018年上半年排名']=999
        if df.loc[i,'2018年下半年']==0:
            df.loc[i,'2018年下半年排名']=999
        if df.loc[i,'2019年上半年']==0:
            df.loc[i,'2019年上半年排名']=999 
    
    df['2019年上半年再贴现发生量得分']=df['2019年上半年排名'].replace(1000,10).replace(999,0).replace(8,800).replace(1,10).replace(2,10).replace(3,8).replace(4,8).replace(5,6).replace(7,6).replace(800,4).replace(9,4).replace(1000,4)
    df['2018年下半年再贴现发生量得分']=df['2018年下半年排名'].replace(1000,10).replace(999,0).replace(8,800).replace(1,10).replace(2,10).replace(3,8).replace(4,8).replace(5,6).replace(7,6).replace(800,4).replace(9,4).replace(1000,4)
    df['2018年上半年再贴现发生量得分']=df['2018年上半年排名'].replace(1000,10).replace(999,0).replace(8,800).replace(1,10).replace(2,10).replace(3,8).replace(4,8).replace(5,6).replace(7,6).replace(800,4).replace(9,4).replace(1000,4)

    

    
    
    df['2019年上半年再贴现发生量环比增长率']=(df['2019年上半年']-df['2018年下半年'])/df['2018年下半年']
    df['2018年下半年再贴现发生量环比增长率']=(df['2018年下半年']-df['2018年上半年'])/df['2018年上半年']

    df= df.reset_index(drop=False)    #重新定义索引

    
    for i in range(0,len(df)):
        if df.loc[i,'2019年上半年再贴现发生量环比增长率']>0.02:
            df.loc[i,'2019年上半年再贴现发生量环比增长率得分']=2
        elif df.loc[i,'2019年上半年再贴现发生量环比增长率']< 0:
            df.loc[i,'2019年上半年再贴现发生量环比增长率得分']=0
        else:
            df.loc[i,'2019年上半年再贴现发生量环比增长率得分']=0
            
  
    
    for i in range(0,len(df)):
        if df.loc[i,'2018年下半年再贴现发生量环比增长率']>0.02:
            df.loc[i,'2018年下半年再贴现发生量环比增长率得分']=2
        elif df.loc[i,'2018年下半年再贴现发生量环比增长率']< 0:
            df.loc[i,'2018年下半年再贴现发生量环比增长率得分']=0
        else:
            df.loc[i,'2018年下半年再贴现发生量环比增长率得分']=0
            
  
    df=df.set_index(['交易机构'])
    df['来源']='经营报表'

    df.to_excel('再贴现分数.xls')



    
    
    
    
zaitiexian()

# 利润得分 

In [5]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def zaitiexian():
    global lirun
    df=pd.read_excel('计算分数/2018年12月银行自营收入统计表考核口径（20190109）2.xls',encoding='GB18030')
    df=df[['地区','市县','（一）票据直贴收入','1.票据转贴收入','银行自营收入']]
    df=df.fillna(method='ffill')  #用缺失值前面的数字填充缺失值
    df=df[df['市县']=='小计']
    df['2018年全年利润']=df['（一）票据直贴收入']+df['1.票据转贴收入']
    df=df.set_index(['地区'])
    df.rename(columns={'银行自营收入':'2018年全年银行自营收入'}, inplace=True)

    
    df=df[['2018年全年利润','2018年全年银行自营收入']]  #########################


    
    df2=pd.read_excel('计算分数/2018年6月银行自营收入统计表.xls','累计',encoding='GB18030')
    df2=df2[['地区','市县','（一）票据直贴收入','1.票据转贴收入','银行自营收入']]
    df2=df2.fillna(method='ffill')  #用缺失值前面的数字填充缺失值
    df2=df2[df2['市县']=='小计']
    df2['2018年上半年利润']=df2['（一）票据直贴收入']+df2['1.票据转贴收入']

    df2=df2.set_index(['地区'])
    df2.rename(columns={'银行自营收入':'2018年上半年银行自营收入'}, inplace=True)
    df2=df2[['2018年上半年利润','2018年上半年银行自营收入']]

 
    df3=pd.read_excel('计算分数/55.xls','累计',encoding='GB18030')
    df3=df3[['地区','市县','（一）票据直贴收入','1.票据转贴收入','银行自营收入']]
    df3=df3.fillna(method='ffill')  #用缺失值前面的数字填充缺失值
    df3=df3[df3['市县']=='小计']
    df3['2019年上半年利润']=df3['（一）票据直贴收入']+df3['1.票据转贴收入']
    df3=df3.set_index(['地区'])
    df3.rename(columns={'银行自营收入':'2019年上半年银行自营收入'}, inplace=True)

    df3=df3[['2019年上半年利润','2019年上半年银行自营收入']]

    lirun=pd.concat([df,df2,df3], axis=1)  
    lirun['2018年下半年利润']=lirun['2018年全年利润']-lirun['2018年上半年利润']
    lirun['2018年下半年银行自营收入']=lirun['2018年全年银行自营收入']-lirun['2018年上半年银行自营收入']


    lirun['2018上半年利润排名']=lirun['2018年上半年利润'].rank(ascending=False).astype('int')
    lirun['2018下半年利润排名']=lirun['2018年下半年利润'].rank(ascending=False).astype('int')
    lirun['2019上半年利润排名']=lirun['2019年上半年利润'].rank(ascending=False).astype('int')
    
    
    
    lirun['2018年上半年银行自营收入排名']=lirun['2018年上半年银行自营收入'].rank(ascending=False).astype('int')
    lirun['2018年下半年银行自营收入排名']=lirun['2018年下半年银行自营收入'].rank(ascending=False).astype('int')
    lirun['2019年上半年银行自营收入排名']=lirun['2019年上半年银行自营收入'].rank(ascending=False).astype('int')
       
    lirun['2018年上半年排名相减']=lirun['2018上半年利润排名']-lirun['2018年上半年银行自营收入排名']
    lirun['2018年下半年排名相减']=lirun['2018下半年利润排名']-lirun['2018年下半年银行自营收入排名']
    lirun['2019年上半年排名相减']=lirun['2019上半年利润排名']-lirun['2019年上半年银行自营收入排名']
    

    
    lirun['2019上半年利润排名得分']=lirun['2019上半年利润排名'].replace(10,1000).replace(1,25).replace(2,21).replace(3,17).replace(4,13).replace(5,10).replace(6,10).replace(7,10).replace(8,10).replace(9,8).replace(1000,8)
    lirun['2018上半年利润排名得分']=lirun['2018上半年利润排名'].replace(10,1000).replace(1,25).replace(2,21).replace(3,17).replace(4,13).replace(5,10).replace(6,10).replace(7,10).replace(8,10).replace(9,8).replace(1000,8)
    lirun['2018下半年利润排名得分']=lirun['2018下半年利润排名'].replace(10,1000).replace(1,25).replace(2,21).replace(3,17).replace(4,13).replace(5,10).replace(6,10).replace(7,10).replace(8,10).replace(9,8).replace(1000,8)



    
    
    lirun['2019上半年利润环比增长率']=(lirun['2019年上半年利润']-lirun['2018年下半年利润'])/lirun['2018年下半年利润']
    lirun['2018下半年利润环比增长率']=(lirun['2018年下半年利润']-lirun['2018年上半年利润'])/lirun['2018年上半年利润']

    
    
    lirun= lirun.reset_index(drop=False)    #重新定义索引

    
    for i in range(0,len(lirun)):
        if lirun.loc[i,'2019上半年利润环比增长率']>0.03:
            lirun.loc[i,'2019上半年利润环比增长率得分']=3
        elif lirun.loc[i,'2019上半年利润环比增长率']<0:
            lirun.loc[i,'2019上半年利润环比增长率得分']=0
        else:
            lirun.loc[i,'2019上半年利润环比增长率得分']=0
            
    for i in range(0,len(lirun)):
        if lirun.loc[i,'2018下半年利润环比增长率']>0.03:
            lirun.loc[i,'2018下半年利润环比增长率得分']=3
        elif lirun.loc[i,'2018下半年利润环比增长率']< 0:
            lirun.loc[i,'2018下半年利润环比增长率得分']=0
        else:
            lirun.loc[i,'2018下半年利润环比增长率得分']=0

            

    for i in range(0,len(lirun)):
        if lirun.loc[i,'2018年上半年排名相减']>=3:
           lirun.loc[i,'2018年上半年排名相减得分']=0
        elif lirun.loc[i,'2018年上半年排名相减']<=-3:
           lirun.loc[i,'2018年上半年排名相减得分']=3
        else:
           lirun.loc[i,'2018年上半年排名相减得分']=0
        
    for i in range(0,len(lirun)):
        if lirun.loc[i,'2018年下半年排名相减']>=3:
           lirun.loc[i,'2018年下半年排名相减得分']=0
        elif lirun.loc[i,'2018年下半年排名相减']<=-3:
           lirun.loc[i,'2018年下半年排名相减得分']=3
        else:
           lirun.loc[i,'2018年下半年排名相减得分']=0
        
    for i in range(0,len(lirun)):
        lirun.loc[i,'地区']= lirun.loc[i,'地区'].replace('地区','市分行')
        if lirun.loc[i,'2019年上半年排名相减']>=3:
           lirun.loc[i,'2019年上半年排名相减得分']=0
        elif lirun.loc[i,'2019年上半年排名相减']<=-3:
           lirun.loc[i,'2019年上半年排名相减得分']=3
        else:
           lirun.loc[i,'2019年上半年排名相减得分']=0    
        
        
        
        
        
    lirun.to_excel('利润分数.xls')
    lirun['来源']='计财数据'
    lirun=lirun.set_index(['地区'])

zaitiexian()

# 汇总 

In [6]:

zong=pd.concat([zongfen,df,lirun], axis=1)  
zong.to_excel('预测分数总.xls')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


# 省行转贴现的利润表分析

In [12]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def zaitiexian():
    global lirun
    df=pd.read_excel('计算分数/2018年12月银行自营收入统计表考核口径（20190109）2.xls',encoding='GB18030')
    df=df[['地区','1.票据转贴收入']]
    df=df[df['地区']=='省分行']
    df=df.set_index(['地区'])
    df.rename(columns={'1.票据转贴收入':'2018年全年利润'}, inplace=True)


    
    df2=pd.read_excel('计算分数/2018年6月银行自营收入统计表.xls','累计',encoding='GB18030')
    df2=df2[['地区','1.票据转贴收入']]
    df2=df2[df2['地区']=='省分行']
    df2=df2.set_index(['地区'])
    df2.rename(columns={'1.票据转贴收入':'2018年上半年利润'}, inplace=True)


    

 
    df3=pd.read_excel('计算分数/55.xls','累计',encoding='GB18030')
    df3=df3[['地区','1.票据转贴收入']]
    df3=df3[df3['地区']=='省分行']
    df3=df3.set_index(['地区'])
    df3.rename(columns={'1.票据转贴收入':'2019年上半年利润'}, inplace=True)

    
    
    
    lirun=pd.concat([df,df2,df3], axis=1)  
    lirun['2018年下半年利润']=lirun['2018年全年利润']-lirun['2018年上半年利润']


    
    
    lirun['2019上半年利润环比增长率']=(lirun['2019年上半年利润']-lirun['2018年下半年利润'])/lirun['2018年下半年利润']
    lirun['2018下半年利润环比增长率']=(lirun['2018年下半年利润']-lirun['2018年上半年利润'])/lirun['2018年上半年利润']

    
    
    lirun= lirun.reset_index(drop=False)    #重新定义索引

    
 
        
        
    lirun['来源']='计财数据'
    lirun=lirun.set_index(['地区'])
    lirun.to_excel('转贴利润.xls')

zaitiexian()